In [1]:
import glob, os
import pandas as pd

In [2]:
# set up file directory and run_id

run_dir = 'C:\\Users\\ywang\\Box\\Modeling and Surveys\\Urban Modeling\\Bay Area UrbanSim\\PBA50\\Final Blueprint runs\\Final Blueprint (s24)\\BAUS v2.2'

os.chdir(run_dir)

In [3]:
# summarize office lump sum acctlog

all_data = pd.DataFrame(columns = ['amount','year','non_residential_sqft', 
                                   'juris', 'tra_id', 'parcel_id', 'description'])
non_residential_sqft = 0

for file in list(glob.glob('*.csv')):
    if 'Office Subsidy Bond_2050.csv' in file:
        df = pd.read_csv(file,
                         usecols = ['amount','year','non_residential_sqft', 
                                    'juris', 'tra_id', 'parcel_id', 'description'])
        print(df.juris[1])
        display(df.head())

        if 'non_residential_sqft' in df.columns:
            sqft = df.non_residential_sqft.sum()
            print('Office sqft: {:,}'.format(int(non_residential_sqft)))
            non_residential_sqft += sqft

            print('Subsidized office sqft by tra type:\n{}'.format(
                df.groupby(['tra_id']).non_residential_sqft.sum()))

            fee_used = (df.loc[df.amount < 0].amount.sum()) * -1
            print('Amount of funding used: {}'.format(fee_used))
            fee_left = df.amount.sum()
            print('Amount of funding left: {}'.format(fee_left))

            first = df.loc[df.non_residential_sqft > 0].year.min()
            print('First round a building is subsidized: {}'.format(first))
            print('Office sqft built by year: \n{}\n'.format(
                  df.loc[df.non_residential_sqft > 0].groupby(['year'])['non_residential_sqft'].sum()))

            all_data = pd.concat([all_data, df])
        
        else:
            print('No subsidized office development.')
            fee_left = df.amount.sum()
            print('Amount of funding left: {} \n'.format(fee_left))
            all_data = pd.concat([all_data, df])

print('Total office sqft created by subsidy: {:,}'.format(int(non_residential_sqft)))
print('Total subsidized office sqft by tra type:\n{}'.format(
        all_data.groupby(['tra_id']).non_residential_sqft.sum()))

In [4]:
# export the aggregated data if needed

all_data.drop(columns = ['Unnamed: 0'], inplace = True)
all_data.to_csv('office_lump_sum_acct_summary.csv', index=False)